In [1]:

#Example, using backtesting library to test crossover strategies

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import json
from pathlib import Path
import alpaca_trade_api as tradeapi
import requests
import os 
from dotenv import load_dotenv

%matplotlib inline


from backtesting.test import SMA




C:\Users\costa\Anaconda3\envs\vizenv\lib\site-packages\backtesting\_plotting.py:44: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
load_dotenv()

alpaca_api_key = os.getenv("Api_key")
alpaca_secret_key = os.getenv("Secret_Key")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')


type(alpaca_api_key)


str

In [3]:
finance_sector = api.alpha_vantage.historic_quotes('XLF', adjusted=True, output_format='pandas')
finance_sector.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2020-05-05,22.16,22.2750,21.7900,21.81,21.81,65120219.0,0.0,1.0
2020-05-04,21.77,21.8902,21.4400,21.86,21.86,44977442.0,0.0,1.0
2020-05-01,22.25,22.3100,21.9115,22.06,22.06,37519690.0,0.0,1.0
2020-04-30,22.91,23.1600,22.6750,22.79,22.79,56730669.0,0.0,1.0
2020-04-29,23.42,23.6550,23.1800,23.38,23.38,70050222.0,0.0,1.0


In [4]:
finance_sector.columns

Index(['1. open', '2. high', '3. low', '4. close', '5. adjusted close',
       '6. volume', '7. dividend amount', '8. split coefficient'],
      dtype='object')

In [5]:
finance_sector.rename(columns={'1. open':'Open','2. high':'High','3. low':'Low', '5. adjusted close':'Close', '6. volume':'Volume'}, inplace=True)
finance_sector.drop(columns=['4. close','7. dividend amount', '8. split coefficient'], inplace=True)
finance_sector

,Open,High,Low,Close,Volume
date,,,,,
2020-05-05,22.16,22.2750,21.7900,21.8100,65120219.0
2020-05-04,21.77,21.8902,21.4400,21.8600,44977442.0
2020-05-01,22.25,22.3100,21.9115,22.0600,37519690.0
2020-04-30,22.91,23.1600,22.6750,22.7900,56730669.0
2020-04-29,23.42,23.6550,23.1800,23.3800,70050222.0
...,...,...,...,...,...
2000-05-09,23.69,23.9700,23.2200,12.7081,569800.0
2000-05-08,22.84,23.5600,22.8400,12.7460,40000.0
2000-05-05,22.69,23.1600,22.6600,12.4970,60500.0


In [6]:
finance_sector.sort_index(ascending=True, inplace=True)
finance_sector

,Open,High,Low,Close,Volume
date,,,,,
2000-05-03,23.78,23.9800,23.0900,12.6106,345500.0
2000-05-04,23.13,23.1900,22.7800,12.4158,146900.0
2000-05-05,22.69,23.1600,22.6600,12.4970,60500.0
2000-05-08,22.84,23.5600,22.8400,12.7460,40000.0
2000-05-09,23.69,23.9700,23.2200,12.7081,569800.0
...,...,...,...,...,...
2020-04-29,23.42,23.6550,23.1800,23.3800,70050222.0
2020-04-30,22.91,23.1600,22.6750,22.7900,56730669.0
2020-05-01,22.25,22.3100,21.9115,22.0600,37519690.0


In [7]:
finance_sector.isnull().sum()
finance_sector.dropna()



,Open,High,Low,Close,Volume
date,,,,,
2000-05-03,23.78,23.9800,23.0900,12.6106,345500.0
2000-05-04,23.13,23.1900,22.7800,12.4158,146900.0
2000-05-05,22.69,23.1600,22.6600,12.4970,60500.0
2000-05-08,22.84,23.5600,22.8400,12.7460,40000.0
2000-05-09,23.69,23.9700,23.2200,12.7081,569800.0
...,...,...,...,...,...
2020-04-29,23.42,23.6550,23.1800,23.3800,70050222.0
2020-04-30,22.91,23.1600,22.6750,22.7900,56730669.0
2020-05-01,22.25,22.3100,21.9115,22.0600,37519690.0


In [8]:
class SmaCross(Strategy):
    def init(self):
        Close = self.data.Close
        self.ma1 = self.I(SMA, Close, 10)
        self.ma2 = self.I(SMA, Close, 20)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


bt = Backtest(finance_sector, SmaCross,
              cash=10000, commission=.002)
bt.run()


Start                     2000-05-03 00:00:00
End                       2020-05-05 00:00:00
Duration                   7307 days 00:00:00
Exposure [%]                          98.9462
Equity Final [$]                      1554.06
Equity Peak [$]                       14896.2
Return [%]                           -84.4594
Buy & Hold Return [%]                 72.9497
Max. Drawdown [%]                    -91.0972
Avg. Drawdown [%]                     -52.713
Max. Drawdown Duration     7160 days 00:00:00
Avg. Drawdown Duration     2419 days 00:00:00
# Trades                                  277
Win Rate [%]                           34.657
Best Trade [%]                        47.0293
Worst Trade [%]                      -22.7831
Avg. Trade [%]                      -0.482892
Max. Trade Duration         128 days 00:00:00
Avg. Trade Duration          27 days 00:00:00
Expectancy [%]                        4.72939
SQN                                  -1.89957
Sharpe Ratio                      

In [9]:
bt.plot()